# Projektarbeit Data Analytics

Sebastian Jana,
Sophie Jana

## Inhaltsverzeichnis
<a id ="inhaltsverzeichnis"></a>

[1. Aufgabe](#aufgabe1)

[2. Aufgabe](#aufgabe2)

[3. Aufgabe](#aufgabe3)

4. Aufgabe
5. Aufgabe

[6. Aufgabe](#aufgabe6)

[7. Quellenverzeichnis](#quellenverzeichnis)




### Aufgabe 1 (Datenvorbereitung)
<a id = "aufgabe1"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)

a) Einlesen der CSV Dateien, welche die Stromerzeugungsdaten und die Börsenstrompreise enthalten, als Dataframe df_hourly.

Nach dem ersten Zusammenführen haben wir festgestellt, dass die Datumsspalte je nach CSV-Datei unterschiedliche Namen für Sommerzeit und Winterzeit hat. Dies führt dazu, dass beim Zusammenführen zwei separate Spalten entstehen. Um dies zu vermeiden, prüfen wir bereits beim Einlesen der Dateien die Spaltennamen und führen sie zu einer einheitlichen Spalte „Datum“ im df_hourly zusammen


In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt


In [ ]:
# Sources for reading csv files from one folder[1],[2]
path = './Daten/Strompreisdaten'
# List all files (.csv) in the given path
csv_files = glob.glob(os.path.join(path, "*csv"))

df_list = []
for i in range(len(csv_files)):
    try:
        df_temp = pd.read_csv(csv_files[i], sep = ",")
        for column in df_temp.columns:
            # Combine the date columns, by getting rid of the naming difference in csv source
            if 'Datum (MESZ)' in column:
                df_temp = df_temp.rename(columns = {'Datum (MESZ)':'DateTime'})
            elif 'Datum (MEZ)' in column:
                df_temp = df_temp.rename(columns = {'Datum (MEZ)':'DateTime'})
        df_list.append(df_temp)
    except Exception as err: 
        print("Fehler beim Einlesen des Files: ", err)
    
df_hourly = pd.concat(df_list)
df_hourly


b) Zur besseren anschließenden Analyse überführen wir die Datumsspalte (vorher dytpe: object) in ein Date-Time-Format. Überprüfung, ob nach der Konvertierung invalide Datumswerte existieren, was nicht der Fall ist.
Alle anderen Spalten haben den dtype float64 und werden so belassen.
Entfernung aller Datensätze, die nicht im Betrachtungszeitraum liegen (2020-2024). Durch Filterung über das Jahr werden 48 Einträge entfernt, da diese nicht in den Betrachtungszeitraum fallen.

In [ ]:
print(df_hourly.shape)
# Converting the column "DateTime" from object to DateTime format
# Invalid values are converted to NaT (Not a Time)
# Source: [3]
df_hourly['DateTime'] = pd.to_datetime(df_hourly['DateTime'], errors="coerce")
print(df_hourly.dtypes)
print(df_hourly['DateTime'].isna().any())


# Create column "Date" from column "DateTime" [4]
df_hourly = df_hourly[(df_hourly['DateTime'].dt.year >= 2020) & (df_hourly['DateTime'].dt.year <= 2024)]

# 48 rows deleted
print(df_hourly.shape) 



c) Beurteilung der Datenqualität und notwenige Datenbereinigungsschritte. 
Qualität der Datumsdaten wurde durch Konvertierung schon überprüft. df_hourly enthält nur Werte für die Jahre 2020-2024. Dataframe enthält keine Null-Werte.

In [ ]:
# Check for missing values
print(df_hourly.isnull().sum())
print(df_hourly.notnull().sum())



d) Erzeugung eines Dataframes df_daily, das für jeden Tag die aggregierten Werte der Leistung der erneuerbaren und nicht erneuerbaren Energiequellen, sowie den täglichen Mittelwert des Börsenpreis enthält. Diese Art der Gruppierung ist sinnvoll für die weitere Analyse, um Trends auf Tagesbasis zu analysieren.

Zur Gruppierung nach Tagen machen wir aus der DateTime nur noch das Datum, indem wir die Uhrzeit entfernen. Anschließend gruppieren wir die Daten nach Datum, wobei die Leistung nicht erneuerbarer und erneuererbarer Energie jeweils für den Tag aufsummiert wird. Zudem gruppieren wir nochmals nach dem Tag und berechnen für jeden Tag den mittleren Börsenpreis. Abschließend joinen wir zu einem gemeinsamen Dataframe df_daily.

In [ ]:
# Grouping by the date without time
# We use a copy to avoid the SettingWithCopyWarning that occurs when we modifiy a view of the original df [5]

df_stunde = df_hourly.copy()

df_stunde['Datum'] = df_stunde['DateTime'].dt.date

df_daily = df_stunde.groupby('Datum')[['Leistung nicht erneuerbar (MW)', 'Leistung erneuerbar (MW)' ]].sum()

df_mean = df_stunde.groupby('Datum')['Day Ahead Auktion Preis (EUR/MWh)'].mean()

df_daily = pd.merge(df_daily, df_mean, on='Datum', how='left').reset_index()

# Change Datum again to datetime format

df_daily['Datum'] = pd.to_datetime(df_daily['Datum'])

df_daily


### Aufgabe 2 (Explorative Analyse der Stromerzeugungs- und Preisdaten)
<a id = "aufgabe2"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)


a) An Welchen 10 Tagen im Betrachtungszeitraum wurden am meisten Strom aus erneuerbaren Energieträgern erzeugt?

Wir beobachten, dass die Tage, an denen der meiste Strom aus erneuererbaren Energieträgern erzeugt wurde, vorwiegend in die Monate Dezember und Februar fallen. 

In [ ]:
ten_highest_days = df_daily.sort_values(by = 'Leistung erneuerbar (MW)', ascending=False).head(10)
ten_highest_days


b) An welchem Tag im Betrachtungszeitraum wurde der höchste Börsenstrompreis verzeichnet und wie hoch war er? (26.08.22)
An welchem Tag im Betrachtungszeitraum wurde der niedrigste Börsenstrompreis verzeichnet und wie hoch war er? (2.7.23)

In [ ]:
highest_price = df_daily['Day Ahead Auktion Preis (EUR/MWh)'].max()
date_highest_price = df_daily[df_daily['Day Ahead Auktion Preis (EUR/MWh)'] == highest_price]

print("Tag mit dem höchsten Börsenstrompreis: " ,date_highest_price['Datum'].dt.date.values[0])
print("Höchster Börsenstrompreis: ", highest_price)

lowest_price = df_daily['Day Ahead Auktion Preis (EUR/MWh)'].min()
date_lowest_price = (df_daily.loc[df_daily['Day Ahead Auktion Preis (EUR/MWh)'] == lowest_price])
print("Tag mit dem niedrigsten Börsenstrompreis: ", date_lowest_price['Datum'].dt.date.values[0])
print("Niedrigster Börsenstrompreis", lowest_price)


c) Wie viele Tage gab es im Betrachtungszeitraum 2020-2024, an denen ein negativer Börsenstrompreis aufgetreten ist?

In [ ]:
negative_price_df = df_daily[df_daily['Day Ahead Auktion Preis (EUR/MWh)'] < 0]
negative_price_day_count = negative_price_df['Day Ahead Auktion Preis (EUR/MWh)'].count()
print("Anzahl der Tage mit einem negativen Börsenstrompreis: ", negative_price_day_count)
negative_price_df


d) Wie viel Strom wurde pro Jahr mit erneuerbaren und mit nicht erneuerbaren Energieträgern erzeugt?

In [ ]:
df_daily['Jahr'] = df_daily['Datum'].astype(str)
df_daily['Jahr'] = df_daily['Jahr'].str[0:4]
df_year = df_daily.groupby('Jahr')[['Leistung nicht erneuerbar (MW)', 'Leistung erneuerbar (MW)']].sum()

#Convert from megawatts (MW) to gigawatts (GW)

df_year['Leistung nicht erneuerbar (GW)'] = (df_year['Leistung nicht erneuerbar (MW)']/1000).round(2)
df_year['Leistung erneuerbar (GW)'] = (df_year['Leistung erneuerbar (MW)']/1000).round(2)
df_year



## Aufgabe 3 (Weiterführende Analyse der Stromerzeugung- und Preisdaten)
<a id = "aufgabe3"></a>


[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)


a) Visualisierung der Verteilung der Börsenstrompreise (insgesamt und pro Jahr)


In [ ]:
# "Line chart to show the distribution of prices per day for the observation period

plt.figure(figsize=(15, 6))
plt.plot(df_daily['Datum'], df_daily['Day Ahead Auktion Preis (EUR/MWh)'], label = 'Line', color = 'blue')
plt.xlabel('Betrachtungszeitrum (2020-2024)')
plt.ylabel('mittlerer Börsenstrompreise in € pro Tag')
plt.title('Mittlerer Börsenstrompreis pro Tag für 2020-2024')

# insgesamt
# den hier 
# Bins auf Tagesebene? (10 Bins)


# pro Jahr: 
# Mean pro Jahr # Boxplot für jedes Jahr
# Analyse: Börsenstrompreis normal informieren (2021, 2023, 2024?) Ukraine, Corona, (Gas)


In [ ]:
import matplotlib.dates as mdates

df_temp_year = df_daily.copy()

# Create modified date for all years with the same year 2000

df_temp_year['Datum'] = df_temp_year['Datum'].astype(str)
df_temp_year['Datum_verändert'] = df_temp_year['Datum'].str.replace(r'\d{4}', '2000', regex=True)

df_temp_year['Datum_verändert'] = pd.to_datetime(df_temp_year['Datum_verändert'])



''''
plt.figure(figsize=(15, 10))
plt.xlabel("Jahr")
plt.ylabel('mittlerer Börsenstrompreise in € pro Tag')
plt.title('Mittlerer Börsenstrompreis pro Tag pro Jahr')
plt.legend(loc='upper right')


# Source [6] Chat-GPT mit Prompt
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())  # Setzt die Haupt-Ticks auf Monatsanfang
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m'))  # Zeigt nur den Monat als Zahl (1-12)

'''
df_temp_year



b) Berechnung auf die einzelnen Jahre des Betrachtungszeitraum verschiedene statistische Kenngrößen für den Börsenstrompreis

In [ ]:

# df_daily contains 'Leistungs nicht erneuerbar' and 'Leistung erneuerbar' as sum for the day 
# and 'Day Ahead Auktion Preis' as mean for the day
df_daily

df_year = df_daily.groupby('Jahr')['Day Ahead Auktion Preis (EUR/MWh)']

df_year.describe()

# maybe round(2)

c) Visualisierung in einem Säulendiagramm die mittleren Börsenstrompreise pro Monat im Betrachtungszeitraum

In [ ]:
df_daily['Monat'] = df_daily['Datum'].dt.month
df_year_monthly = df_daily.groupby(['Jahr', 'Monat'])['Day Ahead Auktion Preis (EUR/MWh)'].mean()

df_year_monthly.plot(kind = 'bar', figsize=(15,10))
plt.ylim(0, 500)

plt.title('Mittlerer Börsenstrompreis pro Monat von 2020-2024')
plt.ylabel('Mittlerer Börsenstrompreis in EUR/MWh')



d)

e) Berechnung und Visualisierung der im Mittel mit erneuerbaren Energieträgern erzeugten Energie im Tagesverlauf, indem wir auf die vollen Stunden eines Tages aggregieren. Der df_daily hat als Aggregation für den Tag die Summe der erneuerbaren und nicht erneuerbaren Energie pro Tag. Deswegen gruppieren wir den df_hourly nochmal nach Tagen und nehmen dann für die erneuerbaren Energien den Mittelwert

In [ ]:
# use df_stunde, which contains the transformed "Datum" as date and not datetime
# df_daily contains 'Leistung erneuerbar (MW) as sum for the day 
# we create new df with 'Leistung erneuerbar (MW) as mean for each day 
df_renewable_daily_mean=df_stunde.groupby('Datum')['Leistung erneuerbar (MW)'].mean().reset_index()
df_renewable_daily_mean['Leistung erneuerbar (GW)'] = df_renewable_daily_mean['Leistung erneuerbar (MW)']/1000

df_renewable_daily_mean



# Also durchschnitt im Tagesverlauf (0 uhr so 1 uhr so mittelwert) nach Uhrzeit groupen (Linie Mittag + Nachmittag mehr)
# Tagesverlauf Preis aus eigenem Interesse? 
# Splitten nach sommer und winter ? Sowohl im Winter als auch sommer Peak (aber nicht so niedrige: Gesamtmenge im Winter höher: Sonnenenergie nicht verteilt, gesteuert)


# einmal recherche zusammensetztung erneuerbare Quellen wann wie produziert in Deutschland!! und nicht erneuerbare (Austieg Atomenergie, Steuerung Bedarf)

f)

In [ ]:
# Heat Map 
# y Jahr 
# x 1 -365 
# wann viel erneuerbar Mitte? 
# rechts achse mit farben 

g) Visualisierung der Zusammensetztung des erzeugten Stroms (erneuerbar vs. nicht erneuerbar) im Zeitverlauf und Analyse der Ergebnisse

In [ ]:
# df_daily contains sum per day for 'Leistung erneuerbar' und 'Leistung nicht erneuerbar'

# or better gigawatt?


df_power_monthly_sum = df_daily.copy()

df_power_monthly_sum = df_power_monthly_sum.groupby(['Jahr', 'Monat'])[['Leistung erneuerbar (MW)', 'Leistung nicht erneuerbar (MW)']].sum()

df_power_monthly_sum


# 20.000 GW weniger da mehr (Gesamtbedarf gleichgeblieben Quelle)
# Prozentuale Darstellung anteil Erneuerbar am gesamt und anteil nicht erneuerbar 

# Stacked bar plot und prozentual an Gesamterzeugung 


### Aufgabe 6 (Analyse von Stromtarif-Angeboten für Endkunden)
<a id = "aufgabe6"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)

a)  Einlesen der JSON-Preisvergleichdaten in einem DataFrame df_cust


In [ ]:

'''
path = './Daten/Endkundenpreise/'

def extract_date_from_filepath(file):
    file = file.replace(path, "")
    # Concatenate the date out of fixed year 2024 and month/day from folder name
    return "2024-" + file[0:5]

def extract_data_from_json(file):
    assert file.endswith(".json"), "Keine json-Datei uebergeben"
    try:
        # Read and load the json file
        df_temp = pd.read_json(json_files[i])
        # Transpose the table: convert the rows to columns
        df_temp = df_temp.T
        # Index = Number of entry in the json file
        df_temp['Angebotsnummer'] = df_temp.index
        # Add date from filename as column
        df_temp['Datum'] = extract_date_from_filepath(file)
        
        return df_temp
    except:
        print("Datei konnte nicht gelesen werden.")


path = './Daten/Endkundenpreise/'
df_list = []
# https://www.tutorialspoint.com/python/os_listdir.htm
for folder in os.listdir(path):
    combined_path = os.path.join(path, folder, "*json")
    json_files = glob.glob(combined_path)

    for i in range(len(json_files)):
        df_temp = extract_data_from_json(json_files[i])
        df_list.append(df_temp)
        
df_cust = pd.concat(df_list)
df_cust
'''

### Quellenverzeichnis
<a id = "quellenverzeichnis"></a>

[1] https://www.geeksforgeeks.org/how-to-read-all-csv-files-in-a-folder-in-pandas/

[2] https://statistikguru.de/python/python-auflisten-dateien-verzeichnis.html

[3] https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

[4] https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.year.html#pandas.Series.dt.year

[5] https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

[6] Chat GPT 

    Prompt (am 05.01.2025): 

    #Using column Jahr, which contains year as string
    # We use a copy to avoid the SettingWithCopyWarning that occurs when we modifiy a view of the original df [5]
    year_2020 = df_daily[df_daily['Jahr'] == '2020'].copy()
    year_2021 = df_daily[df_daily['Jahr'] == '2021'].copy()

    year_2020['Datum'] = year_2020['Datum'].astype(str)
    year_2021['Datum'] = year_2021['Datum'].astype(str)

    # ertste Vorkommen von 2020 auf 2000 setzten

    year_2020['Datum'] = year_2020['Datum'].str.replace('2020', '2000')

    year_2021['Datum'] = year_2021['Datum'].str.replace('2021', '2000')

    year_2020['Datum'] = pd.to_datetime(year_2020['Datum'])
    year_2021['Datum'] = pd.to_datetime(year_2021['Datum'])

    plt.figure(figsize=(12, 6))
    plt.plot(year_2020['Datum'], year_2020['Day Ahead Auktion Preis (EUR/MWh)'], label = '2020', color = 'green')
    plt.plot(year_2021['Datum'], year_2021['Day Ahead Auktion Preis (EUR/MWh)'], label = '2021', color = 'blue')

    Wie kann ich die x-Achse so verändern, dass nicht mehr Jahr-Monat dort steht, sondern nur noch der Monat (also alle Monate von 1-12). Die Darstellung der Tagesdaten soll aber gleich bleiben

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)